# The Kernel

On this demonstration, we are going to show how "The Kernel" from semantic kernel works.

We are covering:

1. Adding Semantic Kernel Latest version to the project
2. Loading the required libraries
3. Defining a settings scope
4. Defining a Greeting Skill
5. Shocasing the difference between the Greeting with a model and a skill
6. Using memory and configuring the system message

#### 1. Loading the Nuget

The latest stable version available (on Feb/2025) is 1.39.0.

On this version, the Agent System is not integrated with other libraries such as AI Foundry SDK neither AutoGen, but it is meant to be later this year.
Good news is, this version already works on top of function calling, enabling better workflow for deterministic actions.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.39.0"

Installed Packages Microsoft.SemanticKernel, 1.39.0

#### 2. Loading the required libraries

You should load the connectors separatelly. It is also a good practice to declare the Types that are being used on the program

In [2]:
using System;
using System.ComponentModel;
using System.IO;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Kernel = Microsoft.SemanticKernel.Kernel;
using KernelPlugin = Microsoft.SemanticKernel.KernelPlugin;
using KernelExtensions = Microsoft.SemanticKernel.KernelExtensions;
using KernelPluginExtensions = Microsoft.SemanticKernel.KernelPluginExtensions;

#### 3. Loading Settings

You should work on environment variables, but for simplicity we are loading from the JSON file.

In [3]:
using System.IO;
using System.Text.Json;

public class Settings
{
    public string Model { get; set; }
    public string AzureEndpoint { get; set; }
    public string ApiKey { get; set; }

    public static (string, string, string) LoadFromFile()
    {
        var jsonString = File.ReadAllText("config/settings.json");
        var options = new JsonSerializerOptions { PropertyNameCaseInsensitive = true };
        var settings = JsonSerializer.Deserialize<Settings>(jsonString, options);
        if (settings == null)
        {
            throw new Exception("Failed to load settings from the configuration file.");
        }
        return (settings.Model, settings.AzureEndpoint, settings.ApiKey);
    }
}

#### 4. Defining a Skill

We define a set of skills to showcase how to choose manually from different available functions, as well as the difference when using the default configured model.

In [4]:
public class GreetingSkill
{
    [KernelFunction, Description("Greet the user based on its name")]
    public string Greetings(string name)
    {
        return $"Hello, {name}! It's nice to meet you!";
    }

    [KernelFunction, Description("Greet the user politely based on its name")]
    public string PoliteGreetings(string name)
    {
        return $"Hello, dear {name}! It's very pleasant to speak with you!";
    }

    [KernelFunction, Description("Greet the user using slang")]
    public string SlangGreetings(string name)
    {
        return $"Yo {name}! Who'd ya doing, daug?";
    }
}

In [5]:
public class UsingModelProgram
{
    public static async Task Ask(string prompt)
    {
        var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();
        IKernelBuilder builder = Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey, serviceId: "4o-mini");

        OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
        {
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        };

        Kernel kernel = builder.Build();
        var importedSkill = kernel.CreatePluginFromObject(new GreetingSkill(), "GreetingSkill");

        var result = await kernel.InvokePromptAsync(prompt, new(openAIPromptExecutionSettings));

        Console.WriteLine(result);
    }
}

In [6]:
public class UsingSkillProgram
{
    public static async Task Ask(string prompt)
    {
        var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();
        IKernelBuilder builder = Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

        Kernel kernel = builder.Build();
        var importedSkill = kernel.CreatePluginFromObject(new GreetingSkill(), "GreetingSkill");

        var arguments = new KernelArguments() { ["name"] = prompt };
        var result = await kernel.InvokeAsync(importedSkill["Greetings"], arguments);
        Console.WriteLine("Normal greeting: \n" + result);
        
        result = await kernel.InvokeAsync(importedSkill["PoliteGreetings"], arguments);
        Console.WriteLine("Polite greeting: \n" + result);

        result = await kernel.InvokeAsync(importedSkill["SlangGreetings"], arguments);
        Console.WriteLine("Slang greeting: \n" + result);
    }
}

#### 5. Greetings using different engines

We are going to show how to use both types of approach when using a task.

In [7]:
Console.WriteLine("This is a test of the Azure OpenAI Chat Completion plugin.");
Console.WriteLine("The first part of the test uses the model directly: \n");

UsingModelProgram.Ask("My name is John Doe.").GetAwaiter().GetResult();

Console.WriteLine("\nThe second part of the test uses a skill: \n");
UsingSkillProgram.Ask("John Doe").GetAwaiter().GetResult();

This is a test of the Azure OpenAI Chat Completion plugin.
The first part of the test uses the model directly: 

Hello, John Doe! How can I assist you today?

The second part of the test uses a skill: 

Normal greeting: 
Hello, John Doe! It's nice to meet you!
Polite greeting: 
Hello, dear John Doe! It's very pleasant to speak with you!
Slang greeting: 
Yo John Doe! Who'd ya doing, daug?


#### 5. Greetings using different engines

You may want to control the availability of the functions on the kernel. In order to orchestrate that, you may use a approach for a factory-like where you change the availability or inclusion of a given function for being called or used.

In [12]:
public class ManagingSkillProgram
{
    private readonly Kernel kernel;
    private readonly OpenAIPromptExecutionSettings executionSettings;
   
    public ManagingSkillProgram()
    {
        var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();
        IKernelBuilder builder = Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
        OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new();

        this.kernel = builder.Build();
        this.executionSettings = openAIPromptExecutionSettings;
        this.executionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();
    }

    private KernelPlugin AddKernelFunction(string skillName, GreetingSkill skill)
    {
        return this.kernel.CreatePluginFromObject(skill, skillName);
    }

    private void RemoveKernelFunction(KernelPlugin skill)
    {
        this.kernel.Plugins.Remove(skill);
        skill = null;
    }

    public async Task Ask(string prompt)
    {
        var arguments = new KernelArguments(executionSettings) { ["name"] = prompt };
        var importedSkill = this.AddKernelFunction("GreetingSkill", new GreetingSkill());
        var result = await this.kernel.InvokeAsync(importedSkill["Greetings"], arguments);
        Console.WriteLine("Normal greeting: \n" + result);
        
        result = await this.kernel.InvokeAsync(importedSkill["PoliteGreetings"], arguments);
        Console.WriteLine("Polite greeting: \n" + result);

        this.RemoveKernelFunction(importedSkill);
        importedSkill = null;

        try
        {
            result = await this.kernel.InvokeAsync(importedSkill["SlangGreetings"], arguments);
            Console.WriteLine("Slang greeting: \n" + result);
        }
        catch (Exception ex)
        {
            Console.WriteLine("\n Error: " + ex.Message);
            Console.WriteLine("\n Plugin was removed. Falling back to model. \n");
            result = await kernel.InvokePromptAsync($"Greet the user properly, calling it by its name, which is {prompt}", arguments);
            Console.WriteLine("model greeting: \n" + result);
        }
    }

    public async Task AskMemory(string prompt)
    {
        var arguments = new KernelArguments(executionSettings) { ["name"] = prompt };
        this.AddKernelFunction("GreetingSkill", new GreetingSkill());

        var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
        var chatHistory = new ChatHistory();

        chatHistory.AddSystemMessage("You are a helpful assistant that properly greets people. If the person name is 'John Doe', you should use the skill 'PoliteGreetings' to greet him politely.");
        chatHistory.AddUserMessage(prompt);

        var chatCompletionResult = await chatCompletionService.GetChatMessageContentAsync(chatHistory, this.executionSettings, this.kernel);

        Console.WriteLine($"Result: {chatCompletionResult}\n");
        chatHistory.AddAssistantMessage(chatCompletionResult.ToString());
        Console.WriteLine($"Chat history: {JsonSerializer.Serialize(chatHistory)}\n");
    }
}

In [13]:
Console.WriteLine("This is a test of the Azure OpenAI Chat Completion plugin.");
Console.WriteLine("We will test the Kernel Plugin management: \n");

var managingSkillProgram = new ManagingSkillProgram();
managingSkillProgram.Ask("John Doe").GetAwaiter().GetResult();

This is a test of the Azure OpenAI Chat Completion plugin.
We will test the Kernel Plugin management: 

Normal greeting: 
Hello, John Doe! It's nice to meet you!
Polite greeting: 
Hello, dear John Doe! It's very pleasant to speak with you!

 Error: Object reference not set to an instance of an object.

 Plugin was removed. Falling back to model. 

model greeting: 
Hello, John Doe! I hope you're doing well. How can I assist you today?


#### 6. Memory and System Message

Finally, to add memory and properly allow the kernel to plan what to do, and use skills accordingly.

In [14]:
Console.WriteLine("This is a test of the Azure OpenAI Chat Completion message with history.");

var managingSkillProgram = new ManagingSkillProgram();
managingSkillProgram.AskMemory("John Doe").GetAwaiter().GetResult();

This is a test of the Azure OpenAI Chat Completion message with history.
Result: Hello, John Doe! It's a pleasure to assist you today. How can I help you?

Chat history: [{"Role":{"Label":"system"},"Items":[{"$type":"TextContent","Text":"You are a helpful assistant that properly greets people. If the person name is \u0027John Doe\u0027, you should use the skill \u0027PoliteGreetings\u0027 to greet him politely."}]},{"Role":{"Label":"user"},"Items":[{"$type":"TextContent","Text":"John Doe"}]},{"Role":{"Label":"assistant"},"Items":[{"$type":"TextContent","Text":"Hello, John Doe! It\u0027s a pleasure to assist you today. How can I help you?"}]}]

